In [1]:
import pickle
import pandas as pd

# data input

In [3]:
df = pd.read_csv("../data/test.csv")
df.head(2)

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70
1,1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26


quitamos las columnas para que este igual el input

In [4]:
df.drop(['depth', 'table'], axis = 1, inplace = True)
df.had(1)

# transformers input

In [6]:
# encoding clarity
with open('../data/modelo/encoding_clarity.pkl', 'rb') as clarity:
    encoding_clarity = pickle.load(clarity)
    
# encoding color
with open('../data/modelo/encoding_color.pkl', 'rb') as color:
    encoding_color = pickle.load(color)
    
# encoding cut
with open('../data/modelo/encoding_cut.pkl', 'rb') as cut:
    encoding_cut = pickle.load(cut)

# estandarización
with open('../data/modelo/estandarizacion.pkl', 'rb') as estandarizacion:
    estandarizacion = pickle.load(estandarizacion)

cargamos el mejor modelo. en este caso, será el gradient boosting:

In [7]:
# cargamos el mejor modelo
with open('../data/modelo/modelo_GradientBoosting.pkl', 'rb') as modelo:
    modelo = pickle.load(modelo)

In [12]:
df['price'] = 0
df.head(1)

,id,carat,cut,color,clarity,x,y,z,price
0,0,0.32,Ideal,I,SI1,4.43,4.49,2.7,0


In [14]:
# estandarizamos todas las columnas que son de tipo numérico usando el transformer que nos guardamos cuando hicimos el modelo

estandarizadas = pd.DataFrame(estandarizacion.transform(df[["carat", "x", "y", "z", 'price']]), columns = ["carat", "x", "y", "z", 'price'])
estandarizadas.drop('price', axis = 1, inplace = True)
estandarizadas.head()

,carat,x,y,z
0,-0.593750,-0.688525,-0.664835,-0.732143
1,0.843750,0.606557,0.571429,0.660714
2,1.500000,1.016393,1.043956,1.053571
3,0.078125,0.136612,0.109890,0.062500
4,1.250000,0.857923,0.796703,1.026786


In [15]:
# hacemos el encoding del resto de las variables

clarity = pd.DataFrame(encoding_clarity.transform(df[["clarity"]]), columns= ["clarity"])
cut = pd.DataFrame(encoding_cut.transform(df[["cut"]]), columns= ["cut"])
color = pd.DataFrame(encoding_color.transform(df[["color"]]), columns= ["color"])

In [16]:
clarity

,clarity
0,7
1,7
2,7
3,5
4,5
...,...
13480,7
13481,7
13482,6
13483,7


In [17]:
df.head(1)

,id,carat,cut,color,clarity,x,y,z,price
0,0,0.32,Ideal,I,SI1,4.43,4.49,2.7,0


In [18]:
df_final = pd.concat([estandarizadas[['carat']], cut, color, clarity, estandarizadas[['x', 'y', 'z']]], axis = 1)
df_final

,carat,cut,color,clarity,x,y,z
0,-0.593750,4,1,7,-0.688525,-0.664835,-0.732143
1,0.843750,3,1,7,0.606557,0.571429,0.660714
2,1.500000,3,2,7,1.016393,1.043956,1.053571
3,0.078125,3,2,5,0.136612,0.109890,0.062500
4,1.250000,0,5,5,0.857923,0.796703,1.026786
...,...,...,...,...,...,...,...
13480,0.625000,3,6,7,0.573770,0.549451,0.428571
13481,0.312500,2,2,7,0.245902,0.274725,0.276786
13482,-0.625000,4,4,6,-0.759563,-0.758242,-0.750000
13483,0.859375,4,1,7,0.721311,0.763736,0.607143


In [21]:
# hacemos la predicción para nuestro nuevo paciente
price = modelo.predict(df_final)
price

array([-0.98420402,  0.44840403,  1.013186  , ..., -0.78874362,
        0.53945284,  0.11949305])

In [22]:
df['price'] = price

In [24]:
df[['id', 'price']].to_csv('../data/submission01.csv', index = False)